In [3]:
    import hopsworks
    import pandas as pd
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    import seaborn as sns
    from matplotlib import pyplot
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    import joblib
    import os

In [5]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/17565
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
iris_fg = fs.get_feature_group(name="iris", version=1)
query = iris_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="iris",
                                  version=1,
                                  description="Read from Iris flower dataset",
                                  labels=["variety"],
                                  query=query)

In [7]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

# Train our model with the Scikit-learn K-nearest-neighbors algorithm using our features (X_train) and labels (y_train)
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train.values.ravel())

Finished: Reading data from Hopsworks, using ArrowFlight (0.84s) 


KNeighborsClassifier(n_neighbors=2)

In [8]:
X_train

sepal_length  sepal_width  petal_length  petal_width
0        5.700000     3.800000      1.700000     0.300000
1        5.600000     2.700000      4.200000     1.300000
3        6.000000     2.900000      4.500000     1.500000
5        6.400000     2.900000      4.300000     1.300000
8        5.200000     4.100000      1.500000     0.100000
..            ...          ...           ...          ...
151      7.790140     2.947908      5.899473     2.032203
152      7.534982     3.074927      5.381092     1.926731
153      7.163176     3.484853      4.971683     1.189200
154      4.797979     4.117785      1.447322     0.541696
156      5.845199     2.724029      3.823924     1.394520

[125 rows x 4 columns]

In [9]:
y_train

variety
0        Setosa
1    Versicolor
3    Versicolor
5    Versicolor
8        Setosa
..          ...
151   Virginica
152   Virginica
153  Versicolor
154      Setosa
156  Versicolor

[125 rows x 1 columns]

In [10]:
# Evaluate model performance using the features from the test set (X_test)
y_pred = model.predict(X_test)

# Compare predictions (y_pred) with the labels in the test set (y_test)
metrics = classification_report(y_test, y_pred, output_dict=True)
results = confusion_matrix(y_test, y_pred)

In [11]:
print(metrics)

{'Setosa': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 12}, 'Versicolor': {'precision': 0.8, 'recall': 1.0, 'f1-score': 0.888888888888889, 'support': 8}, 'Virginica': {'precision': 1.0, 'recall': 0.8333333333333334, 'f1-score': 0.9090909090909091, 'support': 12}, 'accuracy': 0.9375, 'macro avg': {'precision': 0.9333333333333332, 'recall': 0.9444444444444445, 'f1-score': 0.9326599326599326, 'support': 32}, 'weighted avg': {'precision': 0.95, 'recall': 0.9375, 'f1-score': 0.9381313131313131, 'support': 32}}


In [7]:
# Create the confusion matrix as a figure, we will later store it as a PNG image file
df_cm = pd.DataFrame(results, ['True Setosa', 'True Versicolor', 'True Virginica'],
                     ['Pred Setosa', 'Pred Versicolor', 'Pred Virginica'])
cm = sns.heatmap(df_cm, annot=True)
fig = cm.get_figure()

In [8]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir="iris_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/iris_model.pkl")
fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
iris_model = mr.python.create_model(
    name="iris_model", 
    metrics={"accuracy" : metrics['accuracy']},
    model_schema=model_schema,
    description="Iris Flower Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
iris_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/17565/models/iris_model/1


Model(name: 'iris_model', version: 1)